In [1]:
from __future__ import print_function

import os, sys
module_path = os.path.abspath(os.path.join('../../..'))
sys.path.append(module_path)

import numpy as np
import math
import copy
import pandas as pd
from keras.utils import np_utils
from keras.datasets import mnist
import time
import pickle

from pycrcnn.he.HE import TFHEnuFHE
from pycrcnn.he.tfhe_value import TFHEValue
from pycrcnn.he.alu import *

2023-01-21 13:57:24.899842: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-21 13:57:25.491307: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-21 13:57:27.005837: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.2/lib64
2023-01-21 13:57:27.005948: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: can

## Dataset

In [2]:
# Prepare TernaryMNIST Dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Train set
x_train = x_train[:, 6:22, 6:22]

# Create Ternary classification dataset
train_indexes, test_indexes = [], []
for i in range(len(x_train)):
    if y_train[i] == 0 or y_train[i] == 1 or y_train[i] == 2:
        train_indexes.append(i)
for i in range(len(x_test)):
    if y_test[i] == 0 or y_test[i] == 1 or y_test[i] == 2:
        test_indexes.append(i)
x_train = np.subtract(x_train[train_indexes], 128)
x_train.dtype = np.int8
y_train = y_train[train_indexes]

val_images = 5000
idx_train = len(x_train) - val_images
x_train, x_val = x_train[:idx_train], x_train[idx_train:]
y_train, y_val = y_train[:idx_train], y_train[idx_train:]
y_train = np_utils.to_categorical(y_train).astype(int)*16

# Test set
x_test = x_test[:, 6:22, 6:22]
x_test = np.subtract(x_test[test_indexes], 128)
x_test.dtype = np.int8
y_test = y_test[test_indexes]

## Net Architecture

In [3]:
SHRT_MAX = 32767
SHRT_MIN = (-SHRT_MAX - 1 )

def isqrt(n):
    x = n
    y = (x + 1) // 2
    while y < x:
        x = y
        y = (x + n // x) // 2
    return x

In [4]:
# DFA WEIGHTS
def DFA_weights_uniform(in_dim, out_dim):
    range = isqrt((12 * SHRT_MAX) / (in_dim + out_dim))
    return np.random.randint(-range, range, (in_dim, out_dim))

In [5]:
# PLA tanh Activation function
def PLA_tanh(act_in, in_dim, out_dim):
    y_max, y_min = 128, -127
    intervals = [128, 75, 32, -31, -74, -127]
    slopes_inv = [y_max, 8, 2, 1, 2, 8, y_max]

    act_out, act_grad_inv  = np.full((act_in.shape[0], out_dim), y_max), np.full((act_in.shape[0], out_dim), slopes_inv[0])

    for i in range(len(act_in)):
        for j in range(len(act_in[i].squeeze())):
            val = act_in[i].squeeze()[j] // ((1 << 8) * in_dim)
            if val < intervals[0]:
                act_out[i][j] = val // 4
                act_grad_inv[i][j] = slopes_inv[1]
            if val < intervals[1]:
                act_out[i][j] = val
                act_grad_inv[i][j] = slopes_inv[2]
            if val < intervals[2]:
                act_out[i][j] = val * 2
                act_grad_inv[i][j] = slopes_inv[3]
            if val < intervals[3]:
                act_out[i][j] = val
                act_grad_inv[i][j] = slopes_inv[4]
            if val < intervals[4]:
                act_out[i][j] = val // 4
                act_grad_inv[i][j] = slopes_inv[5]
            if val < intervals[5]:
                act_out[i][j] = y_min
                act_grad_inv[i][j] = slopes_inv[6]
    return act_out.astype(int), act_grad_inv

In [6]:
# L2 Loss Function
def L2(y_true, net_out):
    loss = np.zeros((y_true.shape[0], y_true.shape[1]))
    for i in range(len(y_true)):
        for j in range(len(y_true[i])):
            loss[i][j] = net_out[i].squeeze()[j] - y_true[i][j]
    return loss.astype(int)

In [7]:
# Manual matmul used to check overflow
def matmul(m1, m2, check_overflow=False, bits_tfhe = 0, clip_overflow=False, layer=""):
    max_value = 2.**(bits_tfhe-1) - 1.
    min_value = -2.**(bits_tfhe-1)
    res = [[0 for i in range(len(m2[0]))] for j in range(len(m1))]

    for i in range(len(m1)):
        for j in range(len(m2[0])):
            for k in range(len(m2)):
                mul = m1[i][k] * m2[k][j]

                if clip_overflow:
                    mul = max_value if mul>max_value else mul
                    mul = min_value if mul<min_value else mul

                if check_overflow:
                    overflow = 1 if mul>max_value else 0
                    overflow += 1 if mul<min_value else 0
                    if overflow > 0:
                        print("MUL: matmul overflow layer: " + layer)

                res[i][j] += mul

                if clip_overflow:
                    res[i][j] = max_value if res[i][j]>max_value else res[i][j]
                    res[i][j] = min_value if res[i][j]<min_value else res[i][j]

                if check_overflow:
                    overflow = 1 if res[i][j]>max_value else 0
                    overflow += 1 if res[i][j]<min_value else 0
                    if overflow > 0:
                        print("ADD: matmul overflow layer: " + layer)
    return np.array(res)

In [8]:
# MaxPool Layer
class MaxPoolLayer:
    def __init__(self, kernel_size, stride=(1, 1)):
        self.kernel_size = kernel_size
        self.stride = stride

    def forward(self, batch, check_overflow=False, bits_tfhe = 0, clip_overflow=False):
        return np.array([_max(image, self.kernel_size, self.stride) for image in batch])

    def backward(self, loss, lr_inv, check_overflow=False, bits_tfhe = 0, clip_overflow=False):
        return loss

def _max(image, kernel_size, stride):
    x_s = stride[1]
    y_s = stride[0]

    x_k = kernel_size[1]
    y_k = kernel_size[0]

    # print(image)
    x_d = len(image[0])
    y_d = len(image)

    x_o = ((x_d - x_k) // x_s) + 1
    y_o = ((y_d - y_k) // y_s) + 1

    def get_submatrix(matrix, x, y):
        index_row = y * y_s
        index_column = x * x_s
        return matrix[index_row: index_row + y_k, index_column: index_column + x_k]

    return [[np.max(get_submatrix(image, x, y).flatten()) for x in range(0, x_o)] for y in range(0, y_o)]

In [9]:
# Flatten Layer
class FlattenLayer:
    def __init__(self):
        pass

    def forward(self, flatten_in, check_overflow=False, bits_tfhe = 0, clip_overflow=False):
        return flatten_in.reshape(flatten_in.shape[0], flatten_in.shape[1]*flatten_in.shape[2])

    def backward(self, loss, lr_inv, check_overflow=False, bits_tfhe = 0, clip_overflow=False):
        return loss

In [10]:
# FC Layer
class FCLayer:
    def __init__(self, in_dim, out_dim, last_layer = False):
        self.in_dim, self.out_dim = in_dim, out_dim
        self.last_layer = last_layer
        self.weights = np.zeros((in_dim, out_dim)).astype(int)
        self.bias = np.zeros((1, out_dim)).astype(int)
        self.DFA_weights = np.zeros((1, 1)).astype(int)
    
    def forward(self, fc_in, check_overflow=False, bits_tfhe = 0, clip_overflow=False):
        max_value = 2.**(bits_tfhe-1) - 1.
        min_value = -2.**(bits_tfhe-1)
        layer = "fw input="  + repr(self.in_dim)
        self.input = fc_in

        if check_overflow or clip_overflow:
            dot = matmul(self.input, self.weights, check_overflow, bits_tfhe, clip_overflow, layer) + self.bias
        else:
            dot = (self.input @ self.weights) + self.bias

        if clip_overflow:
            dot = np.clip(dot, min_value, max_value)
        
        if check_overflow:
            overflow = (dot[dot>max_value]).size
            overflow += (dot[dot<min_value]).size
            if overflow > 0:
                print("ADD: Bias overflow layer: "  + layer)
                print(overflow)
        
        output, self.act_grad_inv = PLA_tanh(dot, self.in_dim, self.out_dim)
        return output
    
    def backward(self, loss, lr_inv, check_overflow=False, bits_tfhe=0, clip_overflow=False):   
        max_value = 2.**(bits_tfhe-1) - 1.
        min_value = -2.**(bits_tfhe-1)
        layer = "bw input="  + repr(self.in_dim)

        d_DFA = self.compute_dDFA(loss, lr_inv, check_overflow, bits_tfhe, clip_overflow, layer)

        if check_overflow:
            overflow = (d_DFA[d_DFA>max_value]).size
            overflow += (d_DFA[d_DFA<min_value]).size
            if overflow > 0:
                print("Deltas overflow layer: " + layer)
                print(overflow)
        
        if check_overflow or clip_overflow:
            weights_update = matmul(self.input.T, d_DFA, check_overflow, bits_tfhe, clip_overflow, layer)
        else:
            weights_update = self.input.T @ d_DFA
        
        if check_overflow:
            overflow = (weights_update[weights_update>max_value]).size
            overflow += (weights_update[weights_update<min_value]).size
            if overflow > 0:
                print("Weights Update overflow layer: " + layer)
                print(overflow)
        
        weights_update = (weights_update // lr_inv).astype(int)
        self.weights -= weights_update

        if check_overflow:
            overflow = (self.weights[self.weights>max_value]).size
            overflow += (self.weights[self.weights<min_value]).size
            if overflow > 0:
                print("ADD: weights overflow layer: " + layer)
                print(overflow)
        
        ones = np.ones((len(d_DFA), 1)).astype(int)
        if check_overflow or clip_overflow:
            bias_update = matmul(d_DFA.T, ones, check_overflow, bits_tfhe, clip_overflow, layer)
        else:
            bias_update = d_DFA.T @ ones
        
        if check_overflow:
            overflow = (bias_update[bias_update>max_value]).size
            overflow += (bias_update[bias_update<min_value]).size
            if overflow > 0:
                print("Bias Update overflow layer: " + layer)
                print(overflow)
        
        bias_update = (bias_update.T // lr_inv).astype(int)
        self.bias -= bias_update

        if check_overflow:
            overflow = (self.bias[self.bias>max_value]).size
            overflow += (self.bias[self.bias<min_value]).size
            if overflow > 0:
                print("ADD: bias overflow layer: " + layer)
                print(overflow)
        
        return loss
    
    def compute_dDFA(self, loss, lr_inv, check_overflow=False, bits_tfhe=0, clip_overflow=False, layer=""):
        if self.last_layer:
            d_DFA = np.floor_divide(loss, self.act_grad_inv)
        else:
            if self.DFA_weights.shape[0] != loss.shape[1] and  self.DFA_weights.shape[1] != self.weights.shape[1]: # 0 rows, 1 cols
                print("DFA not initialized!")
            if check_overflow or clip_overflow:
                dot = matmul(loss, self.DFA_weights, check_overflow, bits_tfhe, clip_overflow, layer)
            else:
                dot = loss @ self.DFA_weights
            d_DFA = np.floor_divide(dot, self.act_grad_inv)
        return d_DFA

In [11]:
# Network
class Network:
    def __init__(self):
        self.layers = []

    # add layer to network
    def add(self, layer):
        self.layers.append(layer)

    # Test
    def test(self, x_test, y_test, check_overflow=False, bits_tfhe=0, clip_overflow=False):
        corr = 0
        for j in range(len(x_test)):
            pred = self.predict(x_test[j], check_overflow, bits_tfhe, clip_overflow)
            if pred == y_test[j]:
                corr += 1
        return corr / len(x_test) * 100

    # Predict output
    def predict(self, input_data, check_overflow=False, bits_tfhe=0, clip_overflow=False):
        output = np.expand_dims(input_data, axis=0)
        for layer in self.layers:
            output = layer.forward(output, check_overflow, bits_tfhe, clip_overflow)
        return output.argmax()

    # train the network
    def fit(self, x_train, y_train, epochs, batch_size, lr_inv, check_overflow=False, bits_tfhe=0, clip_overflow=False):
        train_accs, val_accs = [], []
        for i in range(epochs):
            epoch_corr = 0
            for j in range(int(len(x_train)/batch_size)):
                batch_corr = 0
                idx_start = j * batch_size
                idx_end = idx_start + batch_size

                batch_in = x_train[idx_start:idx_end]
                batch_target = y_train[idx_start:idx_end]

                # Forward propagation
                for layer in self.layers:
                  batch_in = layer.forward(batch_in, check_overflow, bits_tfhe, clip_overflow)
                fwd_out = batch_in               

                # Loss
                loss = L2(batch_target, fwd_out)

                for r in range(batch_size):
                    if batch_target[r].argmax() == fwd_out[r].argmax():
                        batch_corr += 1
                
                # Backward propagation
                for layer in reversed(self.layers):
                    layer.backward(loss, lr_inv, check_overflow, bits_tfhe, clip_overflow)
                
                epoch_corr += batch_corr

            acc = epoch_corr/len(x_train) * 100
            train_accs.append(acc)
            val_accs.append(self.test(x_val, y_val))
            
        return train_accs, val_accs

## Results

In [12]:
## UPLOAD DFA WEIGHTS
DFA_weights1 = np.load("DFAWeights_L1.npy")
DFA_weights2 = np.load("DFAWeights_L2.npy")

### Encrypted Version

#### Model 1

In [13]:
with open("plain_weights_M1.pkl", "rb") as f:
    weights2M1 = pickle.load(f)
    bias2M1 = pickle.load(f)
    weights3M1 = pickle.load(f)
    bias3M1 = pickle.load(f)
    weights4M1 = pickle.load(f)
    bias4M1 = pickle.load(f)

In [14]:
# Network
net1 = Network()
net1.add(MaxPoolLayer((4, 4), stride=(4, 4)))
net1.add(FlattenLayer())
net1.add(FCLayer(16, 4))
net1.add(FCLayer(4, 2))
net1.add(FCLayer(2, 3, last_layer=True))

net1.layers[2].DFA_weights = DFA_weights1
net1.layers[3].DFA_weights = DFA_weights2

net1.layers[2].weights = weights2M1
net1.layers[2].bias = bias2M1
net1.layers[3].weights = weights3M1
net1.layers[3].bias = bias3M1
net1.layers[4].weights = weights4M1
net1.layers[4].bias = bias4M1

In [16]:
acc = net1.test(x_val, y_val, check_overflow=True, bits_tfhe=22)
print("Val accuracy: " + repr(acc) + " %")

Val accuracy: 77.3 %


#### Model 2

In [22]:
with open("plain_weights_M2.pkl", "rb") as f:
    weights2M2 = pickle.load(f)
    bias2M2 = pickle.load(f)
    weights3M2 = pickle.load(f)
    bias3M2 = pickle.load(f)
    weights4M2 = pickle.load(f)
    bias4M2 = pickle.load(f)

In [23]:
# Network
net2 = Network()
net2.add(MaxPoolLayer((4, 4), stride=(4, 4)))
net2.add(FlattenLayer())
net2.add(FCLayer(16, 4))
net2.add(FCLayer(4, 2))
net2.add(FCLayer(2, 3, last_layer=True))

net2.layers[2].DFA_weights = DFA_weights1
net2.layers[3].DFA_weights = DFA_weights2

net2.layers[2].weights = weights2M2
net2.layers[2].bias = bias2M2
net2.layers[3].weights = weights3M2
net2.layers[3].bias = bias3M2
net2.layers[4].weights = weights4M2
net2.layers[4].bias = bias4M2

In [24]:
acc = net2.test(x_val, y_val, check_overflow=True, bits_tfhe=22)
print("Val accuracy: " + repr(acc) + " %")

Val accuracy: 70.12 %


#### Aggregation

In [31]:
with open("plain_weights_Aggr.pkl", "rb") as f:
    weights2Aggr = pickle.load(f)
    bias2Aggr = pickle.load(f)
    weights3Aggr = pickle.load(f)
    bias3Aggr = pickle.load(f)
    weights4Aggr = pickle.load(f)
    bias4Aggr = pickle.load(f)

In [32]:
# Network
netAggr = Network()
netAggr.add(MaxPoolLayer((4, 4), stride=(4, 4)))
netAggr.add(FlattenLayer())
netAggr.add(FCLayer(16, 4))
netAggr.add(FCLayer(4, 2))
netAggr.add(FCLayer(2, 3, last_layer=True))

netAggr.layers[2].DFA_weights = DFA_weights1
netAggr.layers[3].DFA_weights = DFA_weights2

netAggr.layers[2].weights = weights2Aggr
netAggr.layers[2].bias = bias2Aggr
netAggr.layers[3].weights = weights3Aggr
netAggr.layers[3].bias = bias3Aggr
netAggr.layers[4].weights = weights4Aggr
netAggr.layers[4].bias = bias4Aggr

In [35]:
acc = netAggr.test(x_val, y_val, check_overflow=True, bits_tfhe=22)
print("Val accuracy: " + repr(acc) + " %")

Val accuracy: 86.48 %


In [36]:
acc = netAggr.test(x_test, y_test, check_overflow=True, bits_tfhe=22)
print("Test accuracy: " + repr(acc) + " %")

Test accuracy: 86.68573244359708 %


### Plain Version

In [25]:
# Net 1 structure
net1 = Network()
net1.add(MaxPoolLayer((4, 4), stride=(4, 4)))
net1.add(FlattenLayer())
net1.add(FCLayer(16, 4))
net1.add(FCLayer(4, 2))
net1.add(FCLayer(2, 3, last_layer=True))

net1.layers[2].DFA_weights = DFA_weights1
net1.layers[3].DFA_weights = DFA_weights2

# Net 2 structure
net2 = Network()
net2.add(MaxPoolLayer((4, 4), stride=(4, 4)))
net2.add(FlattenLayer())
net2.add(FCLayer(16, 4))
net2.add(FCLayer(4, 2))
net2.add(FCLayer(2, 3, last_layer=True))

net2.layers[2].DFA_weights = DFA_weights1
net2.layers[3].DFA_weights = DFA_weights2

# Train
train_acc1, val_acc1 = net1.fit(x_train[100:125], y_train[100:125], epochs=3, batch_size=5, lr_inv=256, check_overflow=True, bits_tfhe=22, clip_overflow=False)
train_acc2, val_acc2 = net2.fit(x_train[125:150], y_train[125:150], epochs=3, batch_size=5, lr_inv=256, check_overflow=True, bits_tfhe=22, clip_overflow=False)

print("Validation accuracy net1: " + repr(val_acc1[-1]) + " %" + " LR: 256 BS: 5")
print("Validation accuracy net2: " + repr(val_acc2[-1]) + " %" + " LR: 256 BS: 5")

Validation accuracy net1: 77.3 % LR: 256 BS: 5
Validation accuracy net2: 70.12 % LR: 256 BS: 5


In [26]:
weights2 = [net1.layers[2].weights, net2.layers[2].weights]
bias2 = [net1.layers[2].bias, net2.layers[2].bias]

weights3 = [net1.layers[3].weights, net2.layers[3].weights]
bias3 = [net1.layers[3].bias, net2.layers[3].bias]

weights4 = [net1.layers[4].weights, net2.layers[4].weights]
bias4 = [net1.layers[4].bias, net2.layers[4].bias]

In [27]:
# Net structure
averageNet = Network()
averageNet.add(MaxPoolLayer((4, 4), stride=(4, 4)))
averageNet.add(FlattenLayer())
averageNet.add(FCLayer(16, 4))
averageNet.add(FCLayer(4, 2))
averageNet.add(FCLayer(2, 3, last_layer=True))

average_weights2 = np.mean(weights2, axis=0).astype(int)
average_bias2 = np.mean(bias2, axis=0).astype(int)
average_weights3 = np.mean(weights3, axis=0).astype(int)
average_bias3 = np.mean(bias3, axis=0).astype(int)
average_weights4 = np.mean(weights4, axis=0).astype(int)
average_bias4 = np.mean(bias4, axis=0).astype(int)

averageNet.layers[2].weights = average_weights2
averageNet.layers[2].bias = average_bias2
averageNet.layers[3].weights = average_weights3
averageNet.layers[3].bias = average_bias3
averageNet.layers[4].weights = average_weights4
averageNet.layers[4].bias = average_bias4

acc = averageNet.test(x_val, y_val, check_overflow=True, bits_tfhe=22)
print("Average net accuracy: " + repr(acc) + " %")

Average net accuracy: 86.48 %


In [29]:
acc = averageNet.test(x_test, y_test, check_overflow=True, bits_tfhe=22)
print("Test accuracy: " + repr(acc) + " %")

Test accuracy: 86.68573244359708 %
